# Individual Product Forecasting with Explanatory Variables

In [1]:
import pandas as pd
from prophet import Prophet
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import logging
import itertools
from itertools import product
%matplotlib inline

In [2]:
logging.getLogger().setLevel(logging.ERROR)

In [3]:
def calculate_smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    numerator = np.abs(y_true - y_pred)
    denominator = np.abs(y_true) + np.abs(y_pred)
    
    smape = 100 * np.mean(numerator / (denominator + 1e-10 ))
    return smape

In [4]:
df = pd.read_csv('../dataset/clustered_data.csv')

In [5]:
total_df = df.groupby(['Date', 'Description'], as_index=False)['UnitsSold'].sum()

In [6]:
total_df['lag_1'] = total_df['UnitsSold'].shift(1)
total_df['lag_7'] = total_df['UnitsSold'].shift(7)
total_df['lag_14'] = total_df['UnitsSold'].shift(14)

total_df['Rolling_Mean_7'] = total_df['UnitsSold'].rolling(window=7).mean()
total_df['Rolling_Std_7'] = total_df['UnitsSold'].rolling(window=7).std()

In [7]:
total_df['lag_1'] = total_df['lag_1'].ffill()
total_df['lag_7'] = total_df['lag_7'].ffill()
total_df['lag_14'] = total_df['lag_14'].ffill()

total_df['Rolling_Mean_7'] = total_df['Rolling_Mean_7'].fillna(total_df['UnitsSold'].expanding().mean())

total_df['Rolling_Std_7'] = total_df['Rolling_Std_7'].fillna(total_df['UnitsSold'].expanding().std().fillna(0))


In [8]:
total_df = total_df.rename(columns={'Date': 'date', 'UnitsSold': "y"})

In [9]:
explanatory_df = pd.read_csv('../explanatory_variables/combined_explanatory_variables.csv')

In [10]:
total_df = total_df.merge(explanatory_df, on='date', how='left')

In [11]:
total_df = total_df.rename(columns={'date': 'ds'})
total_df['ds'] = pd.to_datetime(total_df['ds'])

In [12]:
regressors = ['lag_7',
       'Rolling_Mean_7',
       'lag_14',
       'Rolling_Std_7',
       'lag_1',
       'is_Thursday',
       'is_Tuesday']

total_df[regressors] = total_df[regressors].ffill().bfill()

In [13]:
n = len(total_df)
train_end = int(n * 0.6)
val_end = int(n * 0.8)

train_data = total_df.iloc[:train_end]
validation_data = total_df.iloc[train_end:val_end]
test_data = total_df.iloc[val_end:]

In [14]:
clip_threshold = np.percentile(total_df['y'], 99)

# Tuning

In [15]:
param_grid = {
    "changepoint_prior_scale": [0.01, 0.05, 0.1, 0.2, 0.5],
    "seasonality_prior_scale": [0.05, 0.1, 0.5, 1, 5],
    "fourier_order": [3, 5, 10]
}

best_smape = float("inf")
best_params = None
best_model = None

In [16]:
train_data = train_data.replace([np.inf, -np.inf], np.nan).dropna()
validation_data = validation_data.replace([np.inf, -np.inf], np.nan).dropna()
test_data = test_data.replace([np.inf, -np.inf], np.nan).dropna()

In [17]:
products = total_df['Description'].unique()
results = []

In [ ]:
for prod in products:
    print(f"\n🔍 Processing Product: {prod}")
    
    product_df = total_df[total_df['Description'] == prod].copy()
    product_df = product_df.groupby('ds')[['y'] + regressors].sum().reset_index().sort_values('ds')

    if product_df.shape[0] < 30:
        continue

    train_size = int(len(product_df) * 0.6)
    val_size = int(len(product_df) * 0.2)

    train_data = product_df.iloc[:train_size]
    validation_data = product_df.iloc[train_size:train_size + val_size]
    test_data = product_df.iloc[train_size + val_size:]

    best_smape = float("inf")
    best_params = None
    best_model = None

    for cp_scale, seas_scale, fourier in product(param_grid["changepoint_prior_scale"],
                                                 param_grid["seasonality_prior_scale"],
                                                 param_grid["fourier_order"]):
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=cp_scale,
            seasonality_prior_scale=seas_scale
        )

        for reg in regressors:
            model.add_regressor(reg)

        try:
            model.fit(train_data)

            future_val = validation_data[['ds'] + regressors]
            forecast_val = model.predict(future_val)
            forecast_val['yhat'] = np.clip(forecast_val['yhat'], 0, clip_threshold)

            smape_score = calculate_smape(validation_data['y'].values, forecast_val['yhat'].values)

            if smape_score < best_smape:
                best_smape = smape_score
                best_params = (cp_scale, seas_scale, fourier)
                best_model = model

        except Exception as e:
            continue

    if best_model is None:
        continue

    # Store results
    results.append({
        'product': prod,
        'smape': best_smape,
        'best_params': best_params,
        'model': best_model,
        'train_data': train_data,
        'validation_data': validation_data,
        'test_data': test_data
    })


🔍 Processing Product: 12 pencils small tube red spotty


00:04:47 - cmdstanpy - INFO - Chain [1] start processing
00:04:47 - cmdstanpy - INFO - Chain [1] done processing
00:04:48 - cmdstanpy - INFO - Chain [1] start processing
00:04:48 - cmdstanpy - INFO - Chain [1] done processing
00:04:48 - cmdstanpy - INFO - Chain [1] start processing
00:04:48 - cmdstanpy - INFO - Chain [1] done processing
00:04:49 - cmdstanpy - INFO - Chain [1] start processing
00:04:49 - cmdstanpy - INFO - Chain [1] done processing
00:04:49 - cmdstanpy - INFO - Chain [1] start processing
00:04:49 - cmdstanpy - INFO - Chain [1] done processing
00:04:50 - cmdstanpy - INFO - Chain [1] start processing
00:04:50 - cmdstanpy - INFO - Chain [1] done processing
00:04:50 - cmdstanpy - INFO - Chain [1] start processing
00:04:50 - cmdstanpy - INFO - Chain [1] done processing
00:04:50 - cmdstanpy - INFO - Chain [1] start processing
00:04:51 - cmdstanpy - INFO - Chain [1] done processing
00:04:51 - cmdstanpy - INFO - Chain [1] start processing
00:04:51 - cmdstanpy - INFO - Chain [1]


🔍 Processing Product: 12 pencils tall tube posy


00:05:26 - cmdstanpy - INFO - Chain [1] start processing
00:05:26 - cmdstanpy - INFO - Chain [1] done processing
00:05:26 - cmdstanpy - INFO - Chain [1] start processing
00:05:26 - cmdstanpy - INFO - Chain [1] done processing
00:05:27 - cmdstanpy - INFO - Chain [1] start processing
00:05:27 - cmdstanpy - INFO - Chain [1] done processing
00:05:28 - cmdstanpy - INFO - Chain [1] start processing
00:05:28 - cmdstanpy - INFO - Chain [1] done processing
00:05:28 - cmdstanpy - INFO - Chain [1] start processing
00:05:29 - cmdstanpy - INFO - Chain [1] done processing
00:05:29 - cmdstanpy - INFO - Chain [1] start processing
00:05:29 - cmdstanpy - INFO - Chain [1] done processing
00:05:29 - cmdstanpy - INFO - Chain [1] start processing
00:05:29 - cmdstanpy - INFO - Chain [1] done processing
00:05:30 - cmdstanpy - INFO - Chain [1] start processing
00:05:30 - cmdstanpy - INFO - Chain [1] done processing
00:05:30 - cmdstanpy - INFO - Chain [1] start processing
00:05:30 - cmdstanpy - INFO - Chain [1]


🔍 Processing Product: 12 pencils tall tube woodland


00:06:13 - cmdstanpy - INFO - Chain [1] done processing
00:06:13 - cmdstanpy - INFO - Chain [1] start processing
00:06:13 - cmdstanpy - INFO - Chain [1] done processing
00:06:14 - cmdstanpy - INFO - Chain [1] start processing
00:06:14 - cmdstanpy - INFO - Chain [1] done processing
00:06:14 - cmdstanpy - INFO - Chain [1] start processing
00:06:14 - cmdstanpy - INFO - Chain [1] done processing
00:06:15 - cmdstanpy - INFO - Chain [1] start processing
00:06:15 - cmdstanpy - INFO - Chain [1] done processing
00:06:15 - cmdstanpy - INFO - Chain [1] start processing
00:06:15 - cmdstanpy - INFO - Chain [1] done processing
00:06:16 - cmdstanpy - INFO - Chain [1] start processing
00:06:16 - cmdstanpy - INFO - Chain [1] done processing
00:06:16 - cmdstanpy - INFO - Chain [1] start processing
00:06:16 - cmdstanpy - INFO - Chain [1] done processing
00:06:17 - cmdstanpy - INFO - Chain [1] start processing
00:06:17 - cmdstanpy - INFO - Chain [1] done processing
00:06:17 - cmdstanpy - INFO - Chain [1] 


🔍 Processing Product: 6 ribbons rustic charm


00:06:41 - cmdstanpy - INFO - Chain [1] done processing
00:06:41 - cmdstanpy - INFO - Chain [1] start processing
00:06:41 - cmdstanpy - INFO - Chain [1] done processing
00:06:42 - cmdstanpy - INFO - Chain [1] start processing
00:06:42 - cmdstanpy - INFO - Chain [1] done processing
00:06:42 - cmdstanpy - INFO - Chain [1] start processing
00:06:42 - cmdstanpy - INFO - Chain [1] done processing
00:06:43 - cmdstanpy - INFO - Chain [1] start processing
00:06:43 - cmdstanpy - INFO - Chain [1] done processing
00:06:43 - cmdstanpy - INFO - Chain [1] start processing
00:06:43 - cmdstanpy - INFO - Chain [1] done processing
00:06:44 - cmdstanpy - INFO - Chain [1] start processing
00:06:44 - cmdstanpy - INFO - Chain [1] done processing
00:06:44 - cmdstanpy - INFO - Chain [1] start processing
00:06:44 - cmdstanpy - INFO - Chain [1] done processing
00:06:45 - cmdstanpy - INFO - Chain [1] start processing
00:06:45 - cmdstanpy - INFO - Chain [1] done processing
00:06:45 - cmdstanpy - INFO - Chain [1] 


🔍 Processing Product: 60 teatime fairy cake cases


00:07:12 - cmdstanpy - INFO - Chain [1] start processing
00:07:12 - cmdstanpy - INFO - Chain [1] done processing
00:07:12 - cmdstanpy - INFO - Chain [1] start processing
00:07:12 - cmdstanpy - INFO - Chain [1] done processing
00:07:13 - cmdstanpy - INFO - Chain [1] start processing
00:07:13 - cmdstanpy - INFO - Chain [1] done processing
00:07:13 - cmdstanpy - INFO - Chain [1] start processing
00:07:13 - cmdstanpy - INFO - Chain [1] done processing
00:07:13 - cmdstanpy - INFO - Chain [1] start processing
00:07:13 - cmdstanpy - INFO - Chain [1] done processing
00:07:14 - cmdstanpy - INFO - Chain [1] start processing
00:07:14 - cmdstanpy - INFO - Chain [1] done processing
00:07:14 - cmdstanpy - INFO - Chain [1] start processing
00:07:14 - cmdstanpy - INFO - Chain [1] done processing
00:07:14 - cmdstanpy - INFO - Chain [1] start processing
00:07:15 - cmdstanpy - INFO - Chain [1] done processing
00:07:15 - cmdstanpy - INFO - Chain [1] start processing
00:07:15 - cmdstanpy - INFO - Chain [1]

In [ ]:
sorted_results = sorted(results, key=lambda x: x['smape'])

In [ ]:
top_5 = sorted_results[:5]
bottom_5 = sorted_results[-5:]

In [ ]:
def plot_forecasts(product_result, title_prefix):
    product = product_result['product']
    model = product_result['model']
    train_data = product_result['train_data']
    validation_data = product_result['validation_data']
    test_data = product_result['test_data']

    best_validation_forecast = model.predict(validation_data[['ds'] + regressors])
    best_validation_forecast['yhat'] = np.clip(best_validation_forecast['yhat'], 0, clip_threshold)
    best_test_forecast = model.predict(test_data[['ds'] + regressors])
    best_test_forecast['yhat'] = np.clip(best_test_forecast['yhat'], 0, clip_threshold)

    plt.figure(figsize=(14, 6))
    plt.plot(train_data['ds'], train_data['y'], label='Train Actual', color='black')
    plt.plot(validation_data['ds'], validation_data['y'], label='Validation Actual', color='blue')
    plt.plot(test_data['ds'], test_data['y'], label='Test Actual', color='green')
    plt.plot(best_validation_forecast['ds'], best_validation_forecast['yhat'], label='Validation Forecast', color='blue', linestyle='--')
    plt.plot(best_test_forecast['ds'], best_test_forecast['yhat'], label='Test Forecast', color='green', linestyle='--')

    plt.axvline(x=validation_data['ds'].min(), color='gray', linestyle=':', label='Validation Start')
    plt.axvline(x=test_data['ds'].min(), color='gray', linestyle='--', label='Test Start')

    plt.title(f'{title_prefix} - {product}')
    plt.xlabel('Date')
    plt.ylabel('Units Sold')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# Plot top 5
for r in top_5:
    plot_forecasts(r, "Top Performer")

In [ ]:
for r in bottom_5:
    plot_forecasts(r, "Worst Performer")